In [1]:
import sys
sys.path.append('../src')
import time
import os
import numpy as np
import pandas as pd
from emagpy import Problem, EMagPy_version
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display

In [45]:
class App(object):
    def __init__(self):
        
        self.fnameHi = None
        self.fnameLo = None
        self.running = False # True when inverison is running
        self.problem = Problem()
        self.problem.runninUI = True

        # select type of sensors
        def sensorComboFunc(change):
            index = change['new']
            print('sensor selected is:', sensors[index])
            if index == 1 or index == 2:
                showGF(True)
            else:
                showGF(False)
        sensors = ['CMD Mini-Explorer',
                   'CMD Explorer'
                   ]
        sensors = sorted(sensors)
        sensors = ['All'] + sensors
        self.sensorCombo = widgets.Dropdown(
            options=[(s, i) for i, s in enumerate(sensors)]
        )
        self.sensorCombo.observe(sensorComboFunc, names='value')
        self.sensorCombo.layout.width = '15%'
        
        
        # import data
        def importBtnFunc(change):
            fnames = []
            for i, key in enumerate(change['new'].keys()):
                fname = 'output{:d}.csv'.format(i)
                fnames.append(fname)
                with open(fname, 'wb') as f:
                    f.write(change['new'][key]['content'])
            self.processFname(fnames)
            self.replot()
        self.importBtn = widgets.FileUpload(
            accept='.csv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
            multiple=True,  # True to accept multiple files upload else False
            button_style='success',
        )
        self.importBtn.observe(importBtnFunc, names='value')
        
        def importGFLoFunc(change):
            fnames = []
            for i, key in enumerate(change['new'].keys()):
                fname = 'output-lo{:d}.csv'.format(i)
                fnames.append(fname)
                with open(fname, 'wb') as f:
                    f.write(change['new'][key]['content'])
            fname = fnames[0]
            self.fnameLo = fname
            self.importGFLo.description = os.path.basename(fname)
        self.importGFLo = widgets.FileUpload(description='Select Lo', disabled=True)
        self.importGFLo.observe(importGFLoFunc, names='value')
        def importGFHiFunc(change):
            fnames = []
            for i, key in enumerate(change['new'].keys()):
                fname = 'output-hi{:d}.csv'.format(i)
                fnames.append(fname)
                with open(fname, 'wb') as f:
                    f.write(change['new'][key]['content'])
            fname = fnames[0]
            self.fnameHi = fname
            self.importGFHi.description = os.path.basename(fname)
        self.importGFHi = widgets.FileUpload(description='Select Hi', disabled=True)
        self.importGFHi.observe(importGFHiFunc, names='value')
        self.hxLabel = widgets.Label(description='Height above the ground [m]:')
        self.hxEdit = widgets.FloatText(value=0, disabled=True)
        self.hxEdit.layout.width = '7%'
        def importGFApplyFunc(b):
            hx = self.hxEdit.value
            device = sensors[self.sensorCombo.value]
            with self.mwRaw:
                print(hx, device, self.fnameLo, self.fnameHi)
            self.problem.importGF(self.fnameLo, self.fnameHi, device, hx)
            self.infoDump('Surveys well imported')
            self.setupUI()
        self.importGFApply = widgets.Button(description='Import', button_style='warning', disabled=True)
        self.importGFApply.on_click(importGFApplyFunc)
        
        
        def showGF(arg):
            visibles = np.array([True, False, False, False, False, False])
            objs = [self.importBtn, self.importGFLo, self.importGFHi,
                    self.hxLabel, self.hxEdit, self.importGFApply]
            with self.mwRaw:
                if arg is False:
                    for o,v in zip(objs, ~visibles):
                        o.disabled = bool(v)
                else:
                    for o,v in zip(objs, visibles):
                        o.disabled = bool(v)
        
        # projection (only if GPS data are available)
        self.projLabel = widgets.Label('Map CRS:')
        
        # preparing the ~5000 projections:
        self.pcs = pd.read_csv('../src/emagpy/pcs.csv')
        pcs_names = self.pcs['COORD_REF_SYS_NAME'].tolist()
        pcs_names.extend(self.pcs['COORD_REF_SYS_NAME_rev'].tolist())

        self.projEdit = widgets.Combobox(
            placeholder='Choose CRS',
            options=pcs_names,
            ensure_option=True,
            tooltip='Type the CRS projection and then select from the options\nDefault is British National Grid / OSGB 1936'
        )
    
        self.projBtn = widgets.Button(
            description='Convert NMEA',
            tooltip='Convert NMEA string coordinates to EPSG coordinates - select a CRS first',
        )
        self.projBtn.on_click(self.projBtnFunc)
        
        # filtering options
        self.filtLabel = widgets.HTML('<b>Filter Options |</b>')
        
        # display options
        self.showParams = {'index': 0, 'coil':'all', 'contour':False, 'vmin':None,
                           'vmax':None,'pts':False, 'cmap':'viridis_r'} 

        # vmin/vmax filtering
        self.vminfLabel = widgets.Label('Vmin:')
        self.vminfEdit = widgets.FloatText(
            value=0,
            disabled=False,
        )
        self.vminfEdit.layout.width = '7%'
        
        self.vmaxfLabel = widgets.Label('Vmax:')
        self.vmaxfEdit = widgets.FloatText(
            value=50,
            disabled=False
        )
        self.vmaxfEdit.layout.width = '7%'

        def keepApplyBtnFunc(change):
            vmin = self.vminEdit.value
            vmax = self.vmaxEdit.value
            self.problem.filterRange(vmin, vmax)
            self.replot()
        self.keepApplyBtn = widgets.Button(
            description='Apply',
        )
        self.keepApplyBtn.on_click(keepApplyBtnFunc)
        
        # rolling mean
        self.rollingEdit = widgets.IntText(
            value=3,
            description='Window size:'
        )
        self.rollingEdit.layout.width = '15%'
        def rollingBtnFunc(b):
            window = self.rollingEdit.value
            self.problem.rollingMean(window=window)
            self.replot()
        self.rollingBtn = widgets.Button(
            description='Rolling Mean'
        )
        self.rollingBtn.on_click(rollingBtnFunc)
        
#         # manual point killer selection
#         def ptsKillerBtnFunc():
#             self.problem.surveys[self.showParams['index']].dropSelected()
#             self.replot()
#         self.ptsKillerBtn = QPushButton('Delete selected points')
#         self.ptsKillerBtn.clicked.connect(ptsKillerBtnFunc)
#         self.ptsKillerBtn.setEnabled(False)
#         self.ptsKillerBtn.setAutoDefault(True)
        
        
        # display options
        self.displayLabel = widgets.HTML('<b>Display Options |</b>')
        self.displayLabel.layout.width = '11%'
        
        # survey selection (useful in case of time-lapse dataset)
        def surveyComboFunc(change):
            self.showParams['index'] = self.surveyCombo.value
            self.replot()
        self.surveyCombo = widgets.Dropdown(
            options=[('Survey1', 1),('Survey 2', 2)]
        )
        self.surveyCombo.layout.width = '12%'
        self.surveyCombo.observe(surveyComboFunc, names='value')
        
        # coil selection
        def coilComboFunc(index):
            self.showParams['coil'] = self.coilCombo.value
            self.replot()
        self.coilCombo = widgets.Dropdown(
            description='',
            options=['VCP0.32','VCP0.71']
        )
        self.coilCombo.layout.width = '12%'
        self.coilCombo.observe(coilComboFunc, names='value')
        
        # alternative button checkable
        def showRadioFunc(val):
            if val['new'] == 'Map':
                showMapOptions(False)
            else:
                showMapOptions(True)
            self.replot()
        self.mapRadio = widgets.Dropdown(
            options=['Raw','Map']
        )
        self.mapRadio.layout.width = '6%'
        self.mapRadio.observe(showRadioFunc, names='value')
        
        def showMapOptions(arg):
            objs = [self.ptsLabel, self.ptsCheck, self.contourLabel,
                    self.contourCheck, self.cmapCombo, self.psMapExpBtn]
            for o in objs:
                o.disabled = arg
            if arg == True:
                self.coilCombo.options = ['all'] + list(self.coilCombo.options)
            else:
                n = len(self.problem.coils)
                if self.coilCombo.value == n:
                    self.coilCombo.value = n-1
                b = list(self.coilCombo.options)
                b.pop(0)
                self.coilCombo.options = b

        # apply the display vmin/vmax for colorbar or y label                
        self.vminLabel = widgets.Label('Vmin:')
        self.vminEdit = widgets.FloatText(
            value=0
        )
        self.vminEdit.layout.width = '5%'
        
        self.vmaxLabel = widgets.Label('Vmax:')
        self.vmaxEdit = widgets.FloatText(
            value=100
        )
        self.vmaxEdit.layout.width = '5%'
        def applyBtnFunc(b):
            self.showParams['vmin'] = self.vminEdit.value
            self.showParams['vmax'] = self.vmaxEdit.value
            self.replot()
        self.applyBtn = widgets.Button(
            description='Apply'
        )
        self.applyBtn.layout.width = '60px'
        self.applyBtn.on_click(applyBtnFunc)
        
        # select different colormap
        def cmapComboFunc(val):
            self.showParams['cmap'] = val['new']
            self.replot()
        self.cmapCombo = widgets.Dropdown(
            description='',
            options=['viridis', 'viridis_r', 'seismic', 'rainbow', 'jet','jet_r'],
            disabled=True
        )
        self.cmapCombo.layout.width = '10%'
        self.cmapCombo.observe(cmapComboFunc, names='value')


        # allow map contouring using tricontourf()
        self.contourLabel = widgets.Label('Contour:')
        def contourCheckFunc(val):
            self.showParams['contour'] = val['new']
            self.replot()
        self.contourCheck = widgets.Checkbox(
            description='Contour',
            indent=False,
            disabled=True
        )
        self.contourCheck.layout.width = '100px'
        self.contourCheck.observe(contourCheckFunc, names='value')
        
        # show data points on the contoured map
        self.ptsLabel = widgets.Label('Pts:')
        def ptsCheckFunc(val):
            self.showParams['pts'] = val['new']
            self.replot()
        self.ptsCheck = widgets.Checkbox(
            description='pts',
            indent=False,
            disabled=True
        )
        self.ptsCheck.layout.width = '50px'
        self.ptsCheck.observe(ptsCheckFunc, names='value')
        
        # export GIS raster layer
#         def expPsMap():
#             fname, _ = QFileDialog.getSaveFileName(importTab,'Export raster map', self.datadir, 'TIFF (*.tif)')
#             if fname != '':
#                 self.setProjection()
#                 self.problem.saveMap(fname=fname, cmap=self.cmapCombo.currentText()) 
        self.psMapExpBtn = widgets.Button(
            description='Exp. GIS layer',
            tooltip='Export a georeferenced TIFF file to directly be imported in GIS software.\n'
                    'Choose the correct EPSG CRS projection!'
        )
#         self.psMapExpBtn.on_click(expPsMap)

        # display it
        self.mwRaw = widgets.Output(layout={'border': '1px solid black'})
        
        
        # layout        
        topLayout = widgets.HBox([
            self.sensorCombo,
            self.importBtn,
            self.importGFLo,
            self.importGFHi,
            self.hxLabel,
            self.hxEdit,
            self.importGFApply,
            self.projLabel,
            self.projEdit,
            self.projBtn
        ])
        
        filtLayout = widgets.HBox([
            self.filtLabel,
            self.vminfLabel,
            self.vminfEdit,
            self.vmaxfLabel,
            self.vmaxfEdit,
            self.keepApplyBtn,
            self.rollingEdit,
            self.rollingBtn,
#             self.ptsKillerBtn
        ])
        
        midLayout = widgets.HBox([
            self.displayLabel,
            self.surveyCombo,
            self.coilCombo,
            self.mapRadio,
            self.vminLabel,
            self.vminEdit,
            self.vmaxLabel,
            self.vmaxEdit,
            self.applyBtn,
            self.cmapCombo,
#             self.contourLabel,
            self.contourCheck,
#             self.ptsLabel,
            self.ptsCheck,
#             self.psMaxExpBtn
        ])
        
        importTab = widgets.VBox([
            topLayout,
            filtLayout,
            midLayout,
            self.mwRaw
        ])
        
        
        # ================ calibration =================
        
        # import ECa csv (same format, one coil per column)
        def ecaImportBtnFunc(change):
            fnames = []
            for i, key in enumerate(change['new'].keys()):
                fname = 'output-dfeca{:d}.csv'.format(i)
                fnames.append(fname)
                with open(fname, 'wb') as f:
                    f.write(change['new'][key]['content'])
            self.fnameECa = fnames[0]
        self.ecaImportBtn = widgets.FileUpload(
            description='Import ECa',
            accept='.csv',
            multiple=False,
            button_style='',
        )
        self.ecaImportBtn.observe(ecaImportBtnFunc, names='value')
        
        # import EC depth-specific (one depth per column) -> can be from ERT
        def ecImportBtnFunc(change):
            fnames = []
            for i, key in enumerate(change['new'].keys()):
                fname = 'output-dfec{:d}.csv'.format(i)
                fnames.append(fname)
                with open(fname, 'wb') as f:
                    f.write(change['new'][key]['content'])
            self.fnameEC = fnames[0]
        self.ecImportBtn = widgets.FileUpload(
            description='Import EC profiles',
            accept='.csv',
            multiple=False,
            button_style='',
        )
        self.ecImportBtn.observe(ecImportBtnFunc, names='value')
        
        # choose which forward model to use
        forwardCalibs = ['CS', 'FSlin', 'FSeq']
        self.forwardCalibCombo = widgets.Dropdown(
            options=forwardCalibs
        )
        
        # perform the fit (equations display in the console)
        def fitCalibBtnFunc(b):
            forwardModel = self.forwardCalibCombo.value
            self.mwCalib.clear_output()
            with self.mwCalib:
                fig, ax = plt.subplots()
                self.problem.calibrate(fnameECa=self.fnameECa, fnameEC=self.fnameEC,
                           forwardModel=forwardModel, ax=ax)
                plt.show()
        self.fitCalibBtn = widgets.Button(description='Fit calibration')
        self.fitCalibBtn.on_click(fitCalibBtnFunc)
        
        # apply the calibration to the ECa measurements of the survey imported
        def applyCalibBtnFunc(b):
            forwardModel = self.forwardCalibCombo.value
            self.mwCalib.clear_output()
            with self.mwCalib:
                fig, ax = plt.subplots()
                self.problem.calibrate(fnameECa=self.fnameECa, fnameEC=self.fnameEC,
                                    forwardModel=forwardModel, apply=True, ax=ax)
                plt.show()
            self.replot()
            self.infoDump('Calibration applied')
        self.applyCalibBtn = widgets.Button(description='Apply Calibration')
        self.applyCalibBtn.on_click(applyCalibBtnFunc)
        
        
        # graph
        self.mwCalib = widgets.Output()
        
        
        # layout
        calibOptions = widgets.HBox([
            self.ecaImportBtn,
            self.ecImportBtn,
            self.forwardCalibCombo,
            self.fitCalibBtn,
            self.applyCalibBtn
        ])
        calibTab = widgets.VBox([
            calibOptions,
            self.mwCalib
        ])
        
        
        
        # ================== error model ===========================
        
        self.surveyErrCombo = widgets.Dropdown()
        self.coilErrCombo = widgets.Dropdown()
        
        def fitErrBtnFunc(b):
            index = self.surveyErrCombo.value
            coil = self.coilErrCombo.value
            self.mwErr.clear_output()
            self.mwErrMap.clear_output()
            with self.mwErr:
                fig, ax = plt.subplots()
                self.problem.crossOverPointsError(index=index, coil=coil, dump=self.infoDump, ax=ax)
                plt.show()
            with self.mwErrMap:
                fig, ax = plt.subplots()
                self.problem.plotCrossOverMap(index=index, coil=coil, ax=ax)
                plt.show()
        self.fitErrBtn = widgets.Button(description='Fit Error Model based on colocated measurements')
        self.fitErrBtn.on_click(fitErrBtnFunc)
        
        # graph
        self.mwErr = widgets.Output()
        self.mwErrMap = widgets.Output()
        
        # layout
        errOptionLayout = widgets.HBox([
            self.surveyErrCombo,
            self.coilErrCombo,
            self.fitErrBtn
        ])
        errGraphLayout = widgets.HBox([
            self.mwErr,
            self.mwErrMap
        ])
        errTab = widgets.VBox([
            errOptionLayout,
            errGraphLayout
        ])
        
        
        # ======================================= inversion settings
        invSettingsTab = widgets.VBox([])
        
        # ======================================= inversion tab
        
        forwardModels = ['CS', 'FSlin', 'FSeq', 'Q']
        self.forwardCombo = widgets.Dropdown(
            options=forwardModels,
            tooltip='''Choice of forward model:
        CS : Cumulative Sensitivity with minimize solver
        FSlin : Full solution with LIN conversion
        FSeq : Full solution without LIN conversion'''
        )
        self.forwardCombo.layout.width = '5%'

        def methodComboFunc(change):
            val = change['new']
            objs1 = [self.alphaLabel, self.alphaEdit,
                    self.betaLabel, self.betaEdit,
                    self.gammaLabel, self.gammaEdit,
                    self.nitLabel, self.nitEdit,
                    self.parallelCheck]
            objs2 = [self.annSampleLabel, self.annSampleEdit,
                     self.annNoiseLabel, self.annNoiseEdit]
            if val == 'ANN':
                for o in objs1:
                    o.disabled = True
                for o in objs2:
                    o.disabled = False
            else:
                for o in objs1:
                    o.disabled = False
                for o in objs2:
                    o.disabled = True
                if len(self.problem.surveys) > 1:
                    self.gammaLabel.disabled = False
                    self.gammaEdit.disabled = False
                else:
                    self.gammaLabel.disabled = True
                    self.gammaEdit.disabled = True
            
        
        mMinimize = ['L-BFGS-B', 'CG', 'TNC', 'Nelder-Mead']
        mMCMC = ['ROPE', 'SCEUA', 'DREAM', 'MCMC']
        methods = mMinimize + mMCMC + ['ANN', 'Gauss-Newton']
        self.methodCombo = widgets.Dropdown(
            options=methods,
            tooltip='''Choice of solver:
        L-BFGS-B : minimize, faster
        CG : Congugate Gradient, fast
        TNC : Truncated Newton, robust
        Nelder-Mead : more robust
        ROPE : McMC-based
        SCEUA : MCMC-based
        DREAM : MCMC-based
        MCMC : Markov Chain Monte Carlo
        ANN : Artificial Neural Network
        Gauss-Newton : as its name'''
        )
        self.methodCombo.layout.width = '10%'
        self.methodCombo.observe(methodComboFunc, names='value')
        
        self.alphaLabel = widgets.Label(value='Vertical smooth:')
        self.alphaEdit = widgets.FloatText(
            value=0.07,
            tooltip='Vertical smoothing between layers from the same profiles.\n'
                    'Can be determined from the L-Curve in "inversion settings" tab.',
        )
        self.alphaEdit.layout.width = '5%'

        self.betaLabel = widgets.Label(value='Lateral smooth:')
        self.betaEdit = widgets.FloatText(
            value=0.0,
            tooltip='Lateral smoothing between contiguous profiles.\n 0 means no lateral smoothing.'
        )
        self.betaEdit.layout.width = '5%'
        
        self.gammaLabel = widgets.Label(value='Time smooth:', disabled=True)
        self.gammaEdit = widgets.FloatText(
            value=0.0,
            tooltip='Smoothing between the first survey and other surveys.',
            disabled=True,
        )
        self.gammaEdit.layout.width = '5%'
        
        self.lLabel = widgets.Label(value='Regularization:')
        self.lCombo = widgets.Dropdown(
            options=['l2','l1'],
            tooltip='Set to l1 for sharp model and l2 for smooth model.'
        )
        self.lCombo.layout.width = '5%'
        
        self.nitLabel = widgets.Label(value='Nit:')
        self.nitEdit = widgets.IntText(
            value=15,
            tooltip='Maximum Number of Iterations'
        )
        self.nitEdit.layout.width = '5%'
        
        self.parallelCheck = widgets.Checkbox(
            value=False,
            description='Parallel',
            tooltip='If checked, inversion will be run in parallel.',
            indent=False
        )
        
        self.annSampleLabel = widgets.Label(value='Number of samples:')
        self.annSampleEdit = widgets.IntText(
            value=100,
            tooltip='Number of synthetic samples for training the model.',
            disabled=True
        )
        self.annSampleEdit.layout.width = '5%'
        
        self.annNoiseLabel = widgets.Label(value='Noise [%]:')
        self.annNoiseEdit = widgets.FloatText(
            value=0,
            tooltip='Noise in percent to apply on synthetic data for training the network.',
            disabled=True
        )
        self.annNoiseEdit.layout.width = '5%'
        
#         def logTextFunc(arg):
#             text = self.logText.toPlainText()
#             if arg[0] == '\r':
#                 text = text.split('\n')
#                 text.pop() # remove last element
#                 text = '\n'.join(text) + arg
#             else:
#                 text = text + arg
#             self.logText.setText(text)
#             QApplication.processEvents()
#         self.logText = QTextEdit('hello there !')
#         self.logText.setReadOnly(True)
        
        def invertBtnFunc(b):
            if self.running == False:
                self.problem.ikill = False
                self.running = True
                self.invertBtn.description = 'Kill'
                self.invertBtn.button_style = 'danger'
                with self.logInv:
                    print('running...')
            else: # button press while running => killing
                with self.logInv:
                    print('killing')
                self.problem.ikill = True
                return
#              self.logText.clear()

            # collect parameters
#             depths0, conds0, fixedDepths, fixedConds = self.modelTable.getTable()
#             self.problem.setInit(depths0, conds0, fixedDepths, fixedConds)
            depths0=np.linspace(0.01, 2, 7)
            self.problem.setInit(depths0)
            regularization = self.lCombo.value
            alpha = self.alphaEdit.value
            forwardModel = self.forwardCombo.value
            method = self.methodCombo.value
            beta = self.betaEdit.value
            gamma = self.gammaEdit.value
            depths = np.r_[[0], depths0, [-np.inf]]
            nit = self.nitEdit.value
            nsample = self.annSampleEdit.value
            noise = self.annNoiseEdit.value
            njobs = -1 if self.parallelCheck.value else 1
#             self.sliceCombo.clear()
#             for i in range(len(depths)-1):
#                 self.sliceCombo.addItem('{:.2f}m - {:.2f}m'.format(depths[i], depths[i+1]))
#             self.sliceCombo.activated.connect(sliceComboFunc)
            
            # invert
            def logTextFunc(arg):
                print(arg, end='')
            with self.logInv:
                self.problem.invert(forwardModel=forwardModel, alpha=alpha,
                                    dump=logTextFunc, regularization=regularization,
                                    method=method, options={'maxiter':nit},
                                    beta=beta, gamma=gamma, nsample=nsample,
                                    noise=noise/100, njobs=njobs)
            
            # plot results
            if self.problem.ikill == False: # program wasn't killed
                replotProf()
                replotMap()
                with self.mwMisfit:
                    fig, ax = plt.subplots()
                    self.problem.showMisfit(ax=ax)
                    plt.show()
                with self.mwOne2One:
                    fig, ax = plt.subplots()
                    self.problem.showOne2one(ax=ax)
                    plt.show()
            
            # reset button
            self.running = False
            self.problem.ikill = False
            self.invertBtn.description = 'Invert'
            self.invertBtn.button_style = 'success'
                  
        self.invertBtn = widgets.Button(
            description='Invert',
            button_style='success')
        self.invertBtn.on_click(invertBtnFunc)
        
        
        # profile display
        showInvParams = {'index':0, 'vmin':None, 'vmax':None, 
                         'cmap':'viridis_r', 'contour':False}
        
        def replotProf():
            self.mwInv.clear_output()
            with self.mwInv:
                fig, ax = plt.subplots()
                self.problem.showResults(**showInvParams, ax=ax)
                plt.show()            
        
        def cmapInvComboFunc(change):
            showInvParams['cmap'] = self.cmapInvCombo.value
            replotProf()
        invCmaps = ['viridis_r', 'viridis', 'seismic', 'rainbow', 'jet']
        self.cmapInvCombo = widgets.Dropdown(options=invCmaps)
        self.cmapInvCombo.observe(cmapInvComboFunc, names='value')
        
        def surveyInvComboFunc(change):
            index = change['new']
            showInvParams['index'] = index
            replotProf()
        self.surveyInvCombo = widgets.Dropdown()
        self.surveyInvCombo.observe(surveyInvComboFunc, names='value')
        
        self.vminInvLabel = widgets.Label(value='vmin:')
        self.vminInvEdit = widgets.FloatText(value=0)
        self.vminInvEdit.layout.width = '10%'
        
        self.vmaxInvLabel = widgets.Label(value='vmax:')
        self.vmaxInvEdit = widgets.FloatText(value=100)
        self.vmaxInvEdit.layout.width = '10%'
        
        def applyInvBtnFunc():
            vmin = self.vminInvEdit.value
            vmax = self.vmaxInvEdit.value
            showInvParams['vmin'] = vmin
            showInvParams['vmax'] = vmax
            replotProf()
        self.applyInvBtn = widgets.Button(description='Apply')
        self.applyInvBtn.on_click(applyInvBtnFunc)
        
        self.contourInvLabel = widgets.Label(value='Contour:')
        def contourInvCheckFunc(change):
            showInvParams['contour'] = self.contourInvCheck.value
            replotProf()
        self.contourInvCheck = widgets.Checkbox(value=False)
        self.contourInvCheck.observe(contourInvCheckFunc, names='value')
 
#         def saveInvDataBtnFunc():
#             fdir = QFileDialog.getExistingDirectory(invTab, 'Choose directory where to save the files')
#             if fdir != '':
#                 self.problem.saveInvData(fdir)
#         self.saveInvDataBtn = QPushButton('Save Results')
#         self.saveInvDataBtn.clicked.connect(saveInvDataBtnFunc)

        
        
        # for the map
        showInvMapParams = {'index':0, 'islice':0, 'vmin':None, 'vmax':None, 'cmap':'viridis_r'}

        def replotMap():
            self.mwInvMap.clear_output()
            with self.mwInvMap:
                fig, ax = plt.subplots()
                self.problem.showSlice(**showInvMapParams, ax=ax)
                plt.show()
                
        def cmapInvMapComboFunc(change):
            showInvMapParams['cmap'] = self.cmapInvMapCombo.value
            replotMap()
        invMapCmaps = ['viridis_r', 'viridis', 'seismic', 'rainbow', 'jet']
        self.cmapInvMapCombo = widgets.Dropdown(options=invMapCmaps)
        self.cmapInvMapCombo.observe(cmapInvMapComboFunc, names='value')
        
        def surveyInvMapComboFunc(change):
            index = change['new']
            showInvMapParams['index'] = index
            replotMap()
        self.surveyInvMapCombo = widgets.Dropdown()
        self.surveyInvMapCombo.observe(surveyInvMapComboFunc, names='value')
        
        self.vminInvMapLabel = widgets.Label(value='vmin:')
        self.vminInvMapEdit = widgets.FloatText(value=0)
        self.vminInvMapEdit.layout.width = '10%'
        
        self.vmaxInvMapLabel = widgets.Label(value='vmax:')
        self.vmaxInvMapEdit = widgets.FloatText(value=100)
        self.vmaxInvMapEdit.layout.width = '10%'
        
        def applyInvMapBtnFunc():
            vmin = self.vminInvMapEdit.value
            vmax = self.vmaxInvMapEdit.value
            showInvMapParams['vmin'] = vmin
            showInvMapParams['vmax'] = vmax
            replotMap()
        self.applyInvMapBtn = widgets.Button(description='Apply')
        self.applyInvMapBtn.on_click(applyInvMapBtnFunc)

        self.sliceLabel = widgets.Label(value='Layer:')
        def sliceComboFunc(change):
            index = change['new']
            showInvMapParams['islice'] = index
            replotMap()
        self.sliceCombo = widgets.Dropdown()
        self.sliceCombo.observe(sliceComboFunc, names='value')
        
        self.contourInvMapLabel = widgets.Label(value='Contour:')
        def contourInvMapCheckFunc(b):
            showInvMapParams['contour'] = self.contourInvMapCheck.value
            replotMap()
        self.contourInvMapCheck = widgets.Checkbox(value=False)
        self.contourInvMapCheck.observe(contourInvMapCheckFunc, names='value')
        
#         def saveInvMapDataBtnFunc():
#             fdir = QFileDialog.getExistingDirectory(invTab, 'Choose directory where to save the files')
#             if fdir != '':
#                 self.problem.saveInvData(fdir)
#         self.saveInvMapDataBtn = QPushButton('Save Results')
#         self.saveInvMapDataBtn.clicked.connect(saveInvMapDataBtnFunc)
        
        
#         def expInvMap():
#             fname, _ = QFileDialog.getSaveFileName(importTab,'Export raster map', self.datadir, 'TIFF (*.tif)')
#             if fname != '':
#                 self.setProjection()
#                 self.problem.saveSlice(fname=fname, islice=self.sliceCombo.currentIndex(), cmap=self.cmapInvMapCombo.currentText())
#         self.invMapExpBtn = QPushButton('Exp. GIS layer')
#         self.invMapExpBtn.setToolTip('Export a georeferenced TIFF file to directly be imported in GIS software.\n'
#                                      'Choose the correct EPSG CRS projection in the "Importing" tab!')
#         self.invMapExpBtn.clicked.connect(expInvMap)
        
        
        
        # graph or log
        self.logInv = widgets.Output()
        self.mwInv = widgets.Output(layout={'border': '1px solid black'})
        self.mwInvMap = widgets.Output(layout={'border': '1px solid black'})

        
        # layout
        
        invOptions1 = widgets.HBox([
            self.forwardCombo,
            self.methodCombo,
            self.alphaLabel,
            self.alphaEdit,
            self.betaLabel,
            self.betaEdit,
            self.gammaLabel,
            self.gammaEdit,
            self.lLabel,
            self.lCombo,
            self.invertBtn
        ])
        invOptions2 = widgets.HBox([
            self.nitLabel,
            self.nitEdit,
            self.parallelCheck,
            self.annSampleLabel,
            self.annSampleEdit,
            self.annNoiseLabel,
            self.annNoiseEdit,
        ])
        
        
        profOptionsLayout = widgets.HBox([
            self.surveyInvCombo,
            self.vminInvLabel,
            self.vminInvEdit,
            self.vmaxInvLabel,
            self.vmaxInvEdit,
            self.applyInvBtn,
            self.contourInvLabel,
            self.contourInvCheck,
            self.cmapInvCombo,
#             self.saveInvDataBtn,
        ])
        profLayout = widgets.VBox([
            profOptionsLayout,
            self.mwInv
        ])

        mapOptionsLayout = widgets.HBox([
            self.surveyInvMapCombo,
            self.sliceLabel,
            self.sliceCombo,
            self.vminInvMapLabel,
            self.vminInvMapEdit,
            self.vmaxInvMapLabel,
            self.vmaxInvMapEdit,
            self.applyInvMapBtn,
            self.contourInvMapLabel,
            self.contourInvMapCheck,
            self.cmapInvMapCombo,
#             self.saveInvMapDataBtn,
#             self.invMapExpBtn
        ])
        mapLayout = widgets.VBox([
            mapOptionsLayout,
            self.mwInvMap
        ])
        
        graphTabs = widgets.Tab()
        graphTabs.children = [self.logInv, profLayout, mapLayout]
        graphTabs.set_title(0, 'Log')
        graphTabs.set_title(1, 'Profiles')
        graphTabs.set_title(2, 'Slices')
        
        invTab = widgets.VBox([
            invOptions1,
            invOptions2,
            graphTabs
        ])
        
        # ============================================= misfit
        
        self.misfitLabel = widgets.Label(value='Misfit after inversion')
        
        self.mwMisfit = widgets.Output()
        self.mwOne2One = widgets.Output()
        
        # layout
        graphLayout = widgets.HBox([self.mwMisfit, self.mwOne2One])
        postTab = widgets.VBox([self.misfitLabel, graphLayout])
        
        
        # =============================================== about tab

        aboutTab = widgets.HTML('''<h1>About EMagPy</h1>
<p><b>Version: {:s}</b></p>
<p><i>EMagPy is a free and open source software for inversion of 1D electromagnetic data</i></p>
<p>If you encouter any issues or would like to submit a feature request, please raise an issue on our gitlab repository at:</p>
<p><a href="https://gitlab.com/hkex/emagpy/issues">https://gitlab.com/hkex/emagpy/issues</a></p>
<p>EMagPy uses a few Python packages:
<a href="https://numpy.org/">numpy</a>, 
<a href="https://pandas.pydata.org/">pandas</a>,
<a href="https://matplotlib.org/">matplotlib</a>,
<a href="https://scipy.org/index.html">scipy</a>,
<a href="https://joblib.readthedocs.io/en/latest/">joblib</a>,
<a href="https://github.com/thouska/spotpy/">spotpy</a>,
<a href="https://pyproj4.github.io/pyproj/stable/">pyproj</a>,
<a href="https://rasterio.readthedocs.io/en/latest/">rasterio (optional)</a>
</p>
<p><strong>EMagPy's core developers: Guillaume Blanchy and Paul McLachlan.<strong></p>
<p>Contributors: Jimmy Boyd, Sina Saneiyan</p>
'''.format(EMagPy_version))
#<p><b>Citing EMagPy</b>:<br>McLachlan P., Blanchy G. and Binley A. 2020.<br>“paper title”<br>Computers & Geosciences, February, 104423. <a href="https://doi.org/10.1016/j.cageo.2020.104423">https://doi.org/10.1016/j.cageo.2020.104423</a>.</p>
        
        
        # overall app
        tabs = widgets.Tab()
        tabs.children = [importTab, calibTab, errTab, invSettingsTab,
                        invTab, postTab, aboutTab]
        tabs.titles = ['Importing'] # doesn't seem to work
        tabs.set_title(0, 'Importing tab')
        tabs.set_title(1, 'ERT Calibration')
        tabs.set_title(2, 'Error modelling')
        tabs.set_title(3, 'Inversion settings')
        tabs.set_title(4, 'Inversion')
        tabs.set_title(5, 'Misfit')
        tabs.set_title(6, 'About')
        tabs.layout.width = '99%'
        tabs.layout.height = '530px'

        appname = widgets.HTML('<h1>EMagPy v1.0.0 (online beta)</h1>')
        appname.layout.height = '50px'
        self.errorLabel= widgets.HTML('Log will appear here')
        self.app = widgets.AppLayout(header=appname,
                                  left_sidebar=None,
                                  center=tabs,
                                  right_sidebar=None,
                                  footer=self.errorLabel)
        self.app.layout.width = '99%'
        self.app.layout.height = '630px'

        
    def setProjection(self):
        val = self.projEdit.value
        try:
            if any(self.pcs['COORD_REF_SYS_NAME'] == val) is True:
                epsg_code = self.pcs['COORD_REF_SYS_CODE'][self.pcs['COORD_REF_SYS_NAME'] == val].values
            elif any(self.pcs['COORD_REF_SYS_NAME_rev'] == val) is True:
                epsg_code = self.pcs['COORD_REF_SYS_CODE'][self.pcs['COORD_REF_SYS_NAME_rev'] == val].values
            epsgVal = 'EPSG:'+str(epsg_code[0])
            self.problem.setProjection(targetProjection=epsgVal)
        except:
            self.errorDump('CRS projection is not correctly defined - See "Importing" tab')

    def projBtnFunc(self):
        try:
            if self.projEdit.value != '':
                self.setProjection()
            self.problem.convertFromNMEA(targetProjection=self.problem.projection)
            self.replot()
        except Exception as e:
            self.errorDump(e)
    
    def replot(self):
        self.mwRaw.clear_output()
        with self.mwRaw:
            index = self.showParams['index']
            coil = self.showParams['coil']
            contour = self.showParams['contour']
            vmin = self.showParams['vmin']
            vmax = self.showParams['vmax']
            pts = self.showParams['pts']
            cmap = self.showParams['cmap']
            fig, ax = plt.subplots(figsize=(16,5))
            if self.mapRadio.value == 'Map':
                self.problem.showMap(index=index, coil=coil, contour=contour,
                                  vmin=vmin, vmax=vmax, pts=pts, cmap=cmap, ax=ax)
            else:
                self.problem.show(index=index, coil=coil, vmin=vmin, vmax=vmax, ax=ax)
            plt.show()
            
    def processFname(self, fnames):
        self.problem.surveys = [] # empty the list of current survey
        if len(fnames) == 1:
            fname = fnames[0]
            self.importBtn.description = os.path.basename(fname)
            self.problem.createSurvey(fname)
#             self.gammaEdit.setVisible(False)
#             self.gammaLabel.setVisible(False)
        else:
            self.importBtn.description = os.path.basename(fnames[0]) + ' .. ' + os.path.basename(fnames[-1])
#             self.gammaEdit.setVisible(True)
#             self.gammaLabel.setVisible(True)
            self.problem.createTimeLapseSurvey(fnames)
        self.infoDump('Files well imported')
        self.setupUI()
        
    def setupUI(self):
        self.mwRaw.clear_output()
        
        # fill the combobox with survey and coil names
        self.coilErrCombo.options = self.problem.coils
        self.coilCombo.options = ['all'] + self.problem.coils
        snames = [(survey.name, i) for i, survey in enumerate(self.problem.surveys)]
        self.surveyCombo.options = snames
        self.surveyErrCombo.options = snames
#         self.surveyInvCombo.options = snames
#         self.surveyInvMapCombo.options = snames
        
        # set to default values
        self.contourCheck.value = False

        # enable widgets
        if 'Latitude' in self.problem.surveys[0].df.columns:
            try:
                float(survey.df['Latitude'][0]) # coordinates are not string
            except: # coordinates are string
                self.problem.projection = 'EPSG:27700' # a default CRS if user hasn't defined anything
                self.projBtnFunc() # automatically convert NMEA string
                self.projBtn.disabled = False
            self.projEdit.disabled = False
        self.keepApplyBtn.disabled = False
        self.rollingBtn.disabled = False
#         self.ptsKillerBtn.disabled = False
        self.coilCombo.disabled = False
        self.surveyCombo.disabled = False
        self.mapRadio.disabled = False
        self.applyBtn.disabled = False
        self.cmapCombo.disabled = False
        self.contourCheck.disabled = False
        self.ptsCheck.disabled = False
    
            
    def errorDump(self, text, flag=1):
        text = str(text)
        timeStamp = time.strftime('%H:%M:%S')
        if flag == 1: # error in red
            col = 'red'
        else:
            col = 'black'
        self.errorLabel.value = '<i style="color:'+col+'">['+timeStamp+']: '+text+'</i>'

    def infoDump(self, text):
        self.errorDump(text, flag=0)
    
    
app = App()
app.app

AppLayout(children=(HTML(value='<h1>EMagPy v1.0.0 (online beta)</h1>', layout=Layout(grid_area='header', heigh…